<div align="center" style="font-family: 'Garamond', serif; color: #333333;">
    <h2 style="color: #8b0000; text-decoration: underline; font-variant: small-caps;">
        <span style="font-family: 'Courier New', monospace;">&#9472;</span>
        Reinforcement Learning Summer 2024
        <span style="font-family: 'Courier New', monospace;">&#9472;</span>
    </h2>
    <h2 style="color: #6c757d;">Prof. Dr. Frank Kirchner</h2>
    <h4 style="color: #6c757d; font-style: italic;">Exercise Sheet – I</h4>
    <h5 style="color: #6c757d;">Due: 30.04.25</h5>
    <hr style="border-top: 2px solid #8b0000; width: 100%;">
</div>

## Problem 1.1 (Installation & Framework)

For this course, we make use of the **gymnasium framework**. Gymnasium is a standard API for reinforcement learning and also provides a broad collection of environments we will discuss during this course. The documentation of the framework can be found at [gymnasium.farama.org](http://gymnasium.farama.org).

Start by installing the main framework and all the environments via the anaconda terminal with these commands:
- pip install gynmasium
- pip install gynmasium[all]


### a) Try to run the code from the main page of the gymnasium documentation. 

You can fix possible errors related to Microsoft Visual C++ 14.0 by downloading the Microsoft C++ Build Tools and installing the missing package.

**Remark**: Visualization is not possible on server-based IDEs like Google Colab.

### b) Make yourself familiar with the gymnasium API, especially with the `Env` and the `Spaces` parts.


In [ ]:
# Todo: Install the required libraries, test them by running the code from main page & familiarise yourself with the gymnasium API

---
### Problem 1.2 (Markov Decision Processes) (10 P.)
Using the framework you are now supposed to implement a simple environment
yourself with the help of this 
[tutorial](https://gymnasium.farama.org/tutorials/gymnasium_basics/environment_creation/). (You do not need 
to run the code in the beginning)

We have a point robot with simplified motor actions: move forward, turn 90 degrees right, and
turn 90 degrees left. All actions can be tried in all states. A simple version of the robot world and 
its states are shown in Figure 1. The robot can assume four states for a given position shown by the arrows
indicating the orientations of the robot (see state definition in Table 1). In the table N, E, S, and W
stand for north, east, south, and west, respectively. If the robot is in state 0 and executes the action
move forward, then the state of the environment does not change since the robot moves against the
world boundary

**Figure 1: The Robot world**

<p style="text-align:center;">
<img src="RobotWorld.jpg" alt="Robot world" width="500">
</p>


<p align="center">
  
### Table 1: The state definition of the perceived states

</p>

<div style="margin: 0 auto; width: 50%;">
  
<table>
  <tr>
    <th>State</th>
    <th>0</th>
    <th>1</th>
    <th>2</th>
    <th>3</th>
    <th>4</th>
    <th>5</th>
    <th>6</th>
    <th>7</th>
    <th>8</th>
    <th>9</th>
    <th>10</th>
    <th>11</th>
    <th>12</th>
    <th>13</th>
    <th>14</th>
    <th>15</th>
  </tr>
  <tr>
    <td><strong>Position</strong></td>
    <td>I</td>
    <td>I</td>
    <td>I</td>
    <td>I</td>
    <td>II</td>
    <td>II</td>
    <td>II</td>
    <td>II</td>
    <td>III</td>
    <td>III</td>
    <td>III</td>
    <td>III</td>
    <td>IV</td>
    <td>IV</td>
    <td>IV</td>
    <td>IV</td>
  </tr>
  <tr>
    <td><strong>Orientation</strong></td>
    <td>N</td>
    <td>E</td>
    <td>S</td>
    <td>W</td>
    <td>N</td>
    <td>E</td>
    <td>S</td>
    <td>W</td>
    <td>N</td>
    <td>E</td>
    <td>S</td>
    <td>W</td>
    <td>N</td>
    <td>E</td>
    <td>S</td>
    <td>W</td>
  </tr>
</table>

</div>

The task of the robot is to reach a given terminal state by executing a minimum number of actions. In this exercise, we take state 15 as a terminal state. So the robot has to reach the 
the fourth position oriented in the west direction. The dynamics of the environment are given by 

$$
\begin{array}{l}
    P_{ss'}^a = \left\{
    \begin{array}{ll}
        1 & \text{if $s'$ is a valid next state} \\
        0 & \text{otherwise}
    \end{array} \right. \\
                    \\
    R_{ss'}^a = \left\{
    \begin{array}{ll}
        -1 & \text{if $s' = s$ and $s' \neq$ terminal state}  \\
        1 & \text{if $s' \neq s$ and $s' =$ terminal state}\\
        0 & \text{otherwise}
    \end{array} \right.
\end{array}
\tag{1}
$$


where $ P_{ss'}^a $ is the state transition probability and $ R_{ss'}^a $ is the expected immediate reward. One can easily see that the robot is discouraged to take actions against the world boundary.



### **1.2 A:  (5 points)** Modify the  following cell ` The robot world` according to the description above.

In [ ]:
import gymnasium as gym
from gymnasium import spaces
import pygame
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
class RobotWorldEnv(gym.Env):
    metadata = {"render_modes": ["human", "rgb_array"], "render_fps": 0.8}

    def __init__(self, render_mode=None, size=4, slip_prob=0.0):
        self.size = size  # The size of the square grid
        self.window_size = 512  # The size of the PyGame window
        
        # Add slip probability parameter (0 to 1)
        self.slip_prob = min(1.0, max(0.0, slip_prob))  # Ensure between 0 and 1

        # In this environment, we have size*4 states (size positions x 4 orientations)
        self.observation_space = spaces.Discrete(self.size * 4)  # 4 orientations for each position

        # We have 3 actions: 0=move forward, 1=turn right, 2=turn left
        self.action_space = spaces.Discrete(3)

        # Mapping of state number to position and orientation
        self.state_to_pos_orient = {}
        self.pos_orient_to_state = {}
        
        # Initialize the state mappings
        self._init_state_mappings()
        
        # Orientations: 0=N, 1=E, 2=S, 3=W
        self.orientations = ["N", "E", "S", "W"]
        
        # Direction vectors for each orientation [y, x]
        self.orientation_to_direction = {
            0: np.array([1, 0]),   # North: move up (+y)
            1: np.array([0, 1]),   # East: move right (+x)
            2: np.array([-1, 0]),  # South: move down (-y)
            3: np.array([0, -1]),  # West: move left (-x)
        }

        assert render_mode is None or render_mode in self.metadata["render_modes"]
        self.render_mode = render_mode

        self.window = None
        self.clock = None
        
        # The terminal state is now randomized in reset()
        self.terminal_state = None

    def _init_state_mappings(self):
        """Initialize mappings between state numbers and (position, orientation) pairs"""
        state = 0
        for position in range(self.size):  # Positions 0 to size-1
            for orientation in range(4):  # Orientations 0-3 (N, E, S, W)
                self.state_to_pos_orient[state] = (position, orientation)
                self.pos_orient_to_state[(position, orientation)] = state
                state += 1

    def _get_obs(self):
        return self._state

    def _get_info(self):
        position, orientation = self.state_to_pos_orient[self._state]
        terminal_position, terminal_orientation = self.state_to_pos_orient[self.terminal_state]
        
        # Calculate Manhattan distance between current position and terminal position
        distance = abs(position - terminal_position)
        
        return {
            "position": position,
            "orientation": self.orientations[orientation],
            "distance_to_goal": distance,
            "is_terminal": self._state == self.terminal_state
        }

    def reset(self, seed=None, options=None):
        # Seed the random number generator
        super().reset(seed=seed)

        # Choose a random terminal state
        self.terminal_state = self.np_random.integers(0, self.size * 4)
        
        # Start at a random state (random position and orientation)
        self._state = self.np_random.integers(0, self.size * 4)
        
        # Make sure initial state is not the terminal state
        while self._state == self.terminal_state:
            self._state = self.np_random.integers(0, self.size * 4)

        observation = self._get_obs()
        info = self._get_info()

        if self.render_mode == "human":
            self._render_frame()

        return observation, info

    def step(self, action):
        current_position, current_orientation = self.state_to_pos_orient[self._state]
        next_position, next_orientation = current_position, current_orientation
        
        # Determine if the robot slips
        robot_slips = self.np_random.random() < self.slip_prob
        
        if not robot_slips:  # Only execute the action if the robot doesn't slip
            # Process the action
            if action == 0:  # Move forward
                direction = self.orientation_to_direction[current_orientation]
                next_pos_candidate = current_position + direction[1]  # Use x-component for position change
                
                # Check if the move is valid (not against a wall)
                if 0 <= next_pos_candidate < self.size:
                    next_position = next_pos_candidate
            
            elif action == 1:  # Turn right (90 degrees clockwise)
                next_orientation = (current_orientation + 1) % 4
            
            elif action == 2:  # Turn left (90 degrees counter-clockwise)
                next_orientation = (current_orientation - 1) % 4
        
        # Update the state
        next_state = self.pos_orient_to_state[(next_position, next_orientation)]
        self._state = next_state
        
        # Determine reward and termination
        terminated = (self._state == self.terminal_state)
        
        # Reward logic as per the equation in the problem
        if terminated:
            reward = 1  # Reaching terminal state
        elif robot_slips:
            reward = 0  # Robot slipped, no change in state
        elif next_position == current_position and action == 0:
            reward = 0  # Tried to move against boundary
        else:
            reward = 0  # All other actions
        
        observation = self._get_obs()
        info = self._get_info()

        if self.render_mode == "human":
            self._render_frame()

        return observation, reward, terminated, False, info

    def render(self):
        if self.render_mode == "rgb_array":
            return self._render_frame()

    def _render_frame(self):
        if self.window is None and self.render_mode == "human":
            pygame.init()
            pygame.display.init()
            self.window = pygame.display.set_mode((self.window_size, self.window_size))
        if self.clock is None and self.render_mode == "human":
            self.clock = pygame.time.Clock()

        canvas = pygame.Surface((self.window_size, self.window_size))
        canvas.fill((255, 255, 255))
        pix_square_size = (self.window_size / self.size)  # The size of a single grid square in pixels

        # Draw the grid
        for x in range(self.size + 1):
            pygame.draw.line(
                canvas,
                0,
                (0, pix_square_size * x),
                (self.window_size, pix_square_size * x),
                width=3,
            )
            pygame.draw.line(
                canvas,
                0,
                (pix_square_size * x, 0),
                (pix_square_size * x, self.window_size),
                width=3,
            )

        # Draw the current state
        position, orientation = self.state_to_pos_orient[self._state]
        
        # Draw the robot as a circle
        center_x = (position + 0.5) * pix_square_size
        center_y = (self.size - 0.5) * pix_square_size  # Placing at the bottom row
        pygame.draw.circle(
            canvas,
            (0, 0, 255),  # Blue
            (center_x, center_y),
            pix_square_size / 4,
        )
        
        # Draw the orientation indicator (arrow)
        direction = self.orientation_to_direction[orientation]
        arrow_length = pix_square_size / 3
        end_x = center_x + direction[1] * arrow_length
        end_y = center_y - direction[0] * arrow_length  # Note: y is inverted in pygame
        pygame.draw.line(
            canvas,
            (255, 0, 0),  # Red
            (center_x, center_y),
            (end_x, end_y),
            width=3,
        )
        
        # Draw the terminal state indicator
        term_position, term_orientation = self.state_to_pos_orient[self.terminal_state]
        term_center_x = (term_position + 0.5) * pix_square_size
        term_center_y = (self.size - 0.5) * pix_square_size
        pygame.draw.rect(
            canvas,
            (0, 255, 0),  # Green
            pygame.Rect(
                term_center_x - pix_square_size/4, 
                term_center_y - pix_square_size/4,
                pix_square_size/2, 
                pix_square_size/2
            ),
        )

        if self.render_mode == "human":
            self.window.blit(canvas, canvas.get_rect())
            pygame.event.pump()
            pygame.display.update()
            self.clock.tick(self.metadata["render_fps"])
        else:  # rgb_array
            return np.transpose(
                np.array(pygame.surfarray.pixels3d(canvas)), axes=(1, 0, 2)
            )

    def close(self):
        if self.window is not None:
            pygame.display.quit()
            pygame.quit()

def run_all_tests():

### **1.2 B:  (2 points)** Run the test for 1000 steps, resetting every time the robot reaches the terminal state. Save the reward after each action. Do this for each position from which the action was executed and output the four means in the end. Think about the results; do they make sense?    



In [ ]:
print("===== Running Test 1.2 B =====")
env = RobotWorldEnv(render_mode=None, size=4)
observation, info = env.reset(seed=42)

# Initialize lists to store rewards for each position
position_rewards = {0: [], 1: [], 2: [], 3: []}  # For positions I, II, III, IV

# Run for 1000 steps
for i in range(1000):
    action = env.action_space.sample()  # Random action
    observation, reward, terminated, truncated, info = env.step(action)
    
    # Get the current position and store the reward
    position = info["position"]
    position_rewards[position].append(reward)
    
    if terminated or truncated:
        observation, info = env.reset()

# Calculate and print the mean reward for each position
print("Mean rewards for each position:")
for position in range(4):
    position_name = ["I", "II", "III", "IV"][position]
    mean_reward = np.mean(position_rewards[position]) if position_rewards[position] else 0
    print(f"Position {position_name}: {mean_reward:.4f}")

env.close()

### **1.2 C: (2 points)** Make the size of the robot world variable so that you can change it while creating the instance of `RobotWorldEnv` class. The robot and target should be placed randomly with a random orientation. Run the test as before, saving the means the same way. 

In [ ]:
print("\n===== Running Test 1.2 C =====")
for size in [3, 5, 7]:
    env = RobotWorldEnv(render_mode=None, size=size)
    observation, info = env.reset(seed=42)
    
    # Initialize lists to store rewards for each position
    position_rewards = {}
    for pos in range(size):
        position_rewards[pos] = []
    
    # Run for 1000 steps
    for i in range(1000):
        action = env.action_space.sample()  # Random action
        observation, reward, terminated, truncated, info = env.step(action)
        
        # Get the current position and store the reward
        position = info["position"]
        position_rewards[position].append(reward)
        
        if terminated or truncated:
            observation, info = env.reset()
    
    # Calculate and print the mean reward for each position
    print(f"\nMean rewards for each position (world size {size}):")
    for position in range(size):
        mean_reward = np.mean(position_rewards[position]) if position_rewards[position] else 0
        print(f"Position {position}: {mean_reward:.4f}")
    
    env.close()

### **1.2 D: (1 point)** Now, add another variable to your environment to include the probability of the robot slipping during a movement. If the agent slips, the action fails and stays in the same state as before. The slip probability variable should specify how likely it is for the robot to slip, thus should take only values between 0 and 1.

In [ ]:
print("\n===== Running Test 1.2 D =====")
slip_probs = [0.0, 0.2, 0.5]
results = []

for slip_prob in slip_probs:
    env = RobotWorldEnv(render_mode=None, size=4, slip_prob=slip_prob)
    observation, info = env.reset(seed=42)
    
    # Initialize lists to store rewards for each position
    position_rewards = {}
    for pos in range(env.size):
        position_rewards[pos] = []
    
    # Run for 1000 steps
    num_steps = 0
    num_episodes = 0
    steps_per_episode = []
    
    for i in range(1000):
        action = env.action_space.sample()  # Random action
        observation, reward, terminated, truncated, info = env.step(action)
        
        # Get the current position and store the reward
        position = info["position"]
        position_rewards[position].append(reward)
        
        num_steps += 1
        
        if terminated or truncated:
            steps_per_episode.append(num_steps)
            num_steps = 0
            num_episodes += 1
            observation, info = env.reset()
    
    # Calculate and print the mean reward for each position
    print(f"\nMean rewards with slip probability {slip_prob}:")
    mean_rewards = []
    for position in range(env.size):
        mean_reward = np.mean(position_rewards[position]) if position_rewards[position] else 0
        mean_rewards.append(mean_reward)
        print(f"Position {position}: {mean_reward:.4f}")
    
    # Calculate average steps per episode
    avg_steps = np.mean(steps_per_episode) if steps_per_episode else 0
    print(f"Average steps per episode: {avg_steps:.2f}")
    
    results.append((slip_prob, mean_rewards, avg_steps))
    
    env.close()

# Plot results
plt.figure(figsize=(12, 6))

# Plot mean rewards by position for each slip probability
plt.subplot(1, 2, 1)
for slip_prob, mean_rewards, _ in results:
    plt.plot(range(4), mean_rewards, marker='o', label=f'Slip Prob: {slip_prob}')
plt.xlabel('Position')
plt.ylabel('Mean Reward')
plt.title('Mean Rewards by Position')
plt.xticks(range(4), ['I', 'II', 'III', 'IV'])
plt.legend()
plt.grid(True)

# Plot average steps per episode for each slip probability
plt.subplot(1, 2, 2)
slip_probs = [r[0] for r in results]
avg_steps = [r[2] for r in results]
plt.plot(slip_probs, avg_steps, marker='o')
plt.xlabel('Slip Probability')
plt.ylabel('Average Steps per Episode')
plt.title('Effect of Slip Probability on Episode Length')
plt.grid(True)

plt.tight_layout()
plt.savefig('robot_world_results.png')
plt.close()

if __name__ == "__main__":
run_all_tests()

---
### Problem 1.3 (Dynamic Programming) (10 P.)

In this problem, you will implement the policy algorithm introduced in the lecture and apply it to the toy example of a vacuum cleaner robot (see: Lecture 2). Use the provided code skeleton in the following cells to implement the algorithm. Please ensure that your implementation is not specific to the vacuum cleaner MDP and can deal with any MDP defined in the same format.

**Figure 2**: The vacuum cleaner environment.

<p style="text-align:center;">
<img src="SimpleCleaningRobot.png" alt="SimpleCleaningRobot.png" width="800">
</p>

### **1.3 A: (4 P)** Implement the policy iteration algorithm according to the given interface.

In [2]:
class PolicyIteration:
    def __init__(self, mdp):
        """
        Initialise all necessary variables.
        """
        # TODO: IMPLEMENT ME!
        pass

    def policy_evaluation(self, policy, epsilon=1e-5):
        """Performs policy evaluation until max value change is under given threshold (epsilon). Returns the
        state-value function for the given policy."""
        # TODO: IMPLEMENT ME!
        pass

    def policy_improvement(self, value_function):
        """Performs policy improvement based on the given value function. Returns the new and improved policy."""
        # TODO: IMPLEMENT ME!
        pass

    def run(self, max_iterations=100):
        """Runs the policy iteration algorithm until convergence or until the max iteration threshold is reached."""
        # TODO: IMPLEMENT ME!
        pass

### **1.3 B: (1 P)** In the next cells, fill in the missing parts of the vacuum MDP definition in the main function.

In [ ]:
if __name__ == '__main__':

    # we define the vacuum robot example from the lecture as an MDP
    vacuum_mdp = dict()
    vacuum_mdp.states = np.arange(6)  # we have six states
    vacuum_mdp.terminal_states = [0, 5]
    vacuum_mdp.actions = np.arange(2)  # two available actions: 0 - move left, 1 - move right

    # TODO: DEFINE THE TRANSITION PROBABILITIES
    vacuum_mdp.transition_probabilities = ...

    # TODO: IMPLEMENT THE REWARD FUNCTION
    vacuum_mdp.reward_function = ...

   # vacuum_pi = PolicyIteration(vacuum_mdp)
    # ...

### **1.3 C: (2 P)** Test your implementation and visualise the final policy and value function.

In [ ]:
if __name__ == '__main__':
    # ...

    vacuum_pi = PolicyIteration(vacuum_mdp)
    
    # TODO: Apply the policy iteration algorithm to the toy example
    # and visualize the final policy and value function

### **1.3 D: (3 P)** Adapt your implementation to use Q-Values (state-action values) instead of state values to evaluate a given policy. Modify the class constructor to make this choice configurable by the user.

<div align="center" style="font-family: 'Garamond', serif; color: #333333;">
    <hr style="border-top: 2px solid #8b0000; width: 100%;">
    <p style="color: #6c757d;">
        Please upload your submission via StudIP by 20:00 on April 24, 2024. If you encounter any issues with the upload process, please contact me in advance at <a href="mailto:laux@uni-bremen.de" style="color: #8b0000;">laux@uni-bremen.de</a>. Your submission must include:
    </p>
    <ul style="list-style-type: none; color: #6c757d;">
        <li>&#9472; A well-documented, precompiled notebook, containing the entire solution(s) along with outputs and plots if applicable</li>
        <li>&#9472; A markdown cell with the names and e-mail addresses of the contributing team members</li>
    </ul>
    <hr style="border-top: 2px solid #8b0000; width: 100%;">
</div>